In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
#!pip install stable-baselines3[extra]

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached importlib_resources-5.12.0-py3-none-any.whl (36 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446680 sha256=5b529d2b4a5ee4cd9fa4194508812534193a9bdd472b18166542138e7c7ddc29
  Stored in directory: c:\users\11447\appdata\local\pip\cache\wheels\04\58\e9\8031ac97e7a2042ddb15456614b07ab978bf41e9edb39783a4
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling Ma

In [3]:
#pip install gym

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827637 sha256=a9c2fb80563467bb59b9cb9d5172781f1a0cf4769646c7880516fa5caa62f9bf
  Stored in directory: c:\users\11447\appdata\local\pip\cache\wheels\4d\c8\dc\d08577bffa680f083f04448e81e4f176d0e45a13eb16f93c0c
Successfully built gym
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gym 
from stable_baselines3 import PPO  # import an algorithm
from stable_baselines3.common.vec_env import DummyVecEnv # import a wrapper
from stable_baselines3.common.evaluation import evaluate_policy # import a wrapper for evaluation

# 2. Load Environment

In [5]:
environment_name = "CartPole-v0"

In [6]:
# Create the environment
env = gym.make(environment_name)

e:\anaconda\anaconda-install\envs\urbsim\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "


In [10]:
episodes = 5 # episode: one full game
for episode in range(1, episodes+1):
    state = env.reset()  # set the environment to initial state
    done = False
    score = 0 
    
    while not done:
        env.render() # view the environment
        action = env.action_space.sample() # generate a random action
        n_state, reward, done, info = env.step(action) # perform the action
        score+=reward # update the score
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

e:\anaconda\anaconda-install\envs\urbsim\lib\site-packages\gym\envs\classic_control\cartpole.py:212: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  "You are calling render method without specifying any render mode. "


ValueError: too many values to unpack (expected 4)

# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [11]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [12]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 4.1600966e+00,  2.3186252e+38, -3.5029808e-01, -8.3706835e+36],
      dtype=float32)

# 3. Train an RL Model

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

In [ ]:
model.learn(total_timesteps=20000)

# 4. Save and Reload Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

# 4. Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 5. Test Model

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

In [ ]:
env.close()

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()